In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import win32com.client
import os
import PySimpleGUI as sg
from reportlab.pdfgen import canvas 
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
import shutil
from shareplum import Site
from requests_ntlm import HttpNtlmAuth
print=sg.Print
menu_def = [      
                ['Ajuda', 'Instruções'], ]    
layout = [
        [sg.Menu(menu_def, tearoff=False)],
        [sg.Text('Insira os arquivos abaixo no formato xlsx :',size=(35, 1))],
        [sg.Text('Login', size=(5, 1), auto_size_text=False, justification='left'),sg.Input(size=(18, 1),key= 'login')],
        [sg.Text('Senha', size=(5, 1), justification='left'),sg.InputText(size=(18, 1),key= 'senha')],
        [sg.Text('Pasta Comprovantes', size=(15, 1), auto_size_text=False, justification='right'),
        sg.InputText(key= 'diretorio'),sg.FileBrowse('Selecionar')],
        [sg.Text('_'  * 80)], 
        [sg.Button('Separar Comprovantes'),sg.Button('Disponibilizar Comprovantes')]
]


window = sg.Window('Disponibilização de comprovantes', layout)
event, values = window.read()
diretorio = values['diretorio']
login = values['login']
senha = values['senha']

while True:
    event, value = window.read()
    if event == 'Separar Comprovantes':
        inicio = ('  Horário de início {}:{} - {}/{}/{} '.format(datetime.now().hour, datetime.now().minute,datetime.now().day,datetime.now().month,datetime.now().year ))
        print(inicio)
        print('[+] Importando dados do Sharepoint, por favor aguarde...')

        cred = HttpNtlmAuth(login, senha)
        site = Site('http://sharepoint/local', auth=cred)
        site2 = Site('http://sharepoint/local', auth=cred)
        site3 = Site('http://sharepoint/local', auth=cred)
        site4 = Site('http://sharepoint/local', auth=cred)
        
        colunas = 'ID','CÓD. DE BARRAS','DATA DE PGTO','Nº GUIAS','EMPRESA'


        lista_tributos = site.List('Guia Digital') # this creates SharePlum object
        lista_demais_areas = site.List('DEMAIS ÁREAS')
        lista_patrimonio = site2.List('Patrimonio')
        lista_pppagamento = site3.List('Liberacao PP')
        lista_excecoes = site4.List('Prévia')

        data_tributos = lista_tributos.GetListItems(fields=['ID','CÓD. DE BARRAS','DATA DE PGTO','Nº GUIAS','EMPRESA']) # this will retrieve all items from list
        data_demais_areas = lista_demais_areas.GetListItems(fields=['ID','CÓD. DE BARRAS','DATA DE PGTO','Nº GUIAS','EMPRESA'])
        data_patrimonio = lista_patrimonio.GetListItems(fields=['ID','CÓD. DE BARRAS','DATA DE PGTO','Nº GUIAS','EMPRESA'])


        df_tributos = pd.DataFrame(data_tributos[0:])
        df_dma = pd.DataFrame(data_demais_areas[0:])
        df_patrimonio = pd.DataFrame(data_patrimonio[0:])

        df_tributos.insert(loc=3, column = 'Area', value = 'Tributos')
        df_patrimonio.insert(loc=3, column = 'Area', value = 'Patrimonio')
        df_dma.insert(loc=3, column = 'Area', value = 'Demais Áreas')
        df_tributos = df_tributos[['ID', 'EMPRESA', 'DATA DE PGTO','Nº GUIAS','CÓD. DE BARRAS','Area']]
        df_patrimonio = df_patrimonio[['ID', 'EMPRESA','DATA DE PGTO','Nº GUIAS' ,'CÓD. DE BARRAS','Area']]
        df_dma = df_dma[['ID', 'EMPRESA', 'DATA DE PGTO','Nº GUIAS','CÓD. DE BARRAS','Area']]
        
        df = pd.concat([df_tributos,df_patrimonio,df_dma])
        df = df.rename(columns={'CÓD. DE BARRAS': 'COD','DATA DE PGTO': 'DATAPGTO','Nº GUIAS':'NGUIAS'})
        hoje = (str(datetime.now().year) + '-' + str(datetime.now().month).zfill(2) + '-' + str(datetime.now().day).zfill(2))
        df = df.loc[df['DATAPGTO'] == hoje ]
        df = df.reset_index (drop = True)
     
        caminho_padrao = (diretorio.split("/"))
        del caminho_padrao[-1]
        separator = '/'
        caminho_padrao = separator.join(caminho_padrao) + separator
        arquivos = os.listdir(caminho_padrao)
        #trocar
        try:
            os.mkdir(caminho_padrao + 'comprovantes')
        except:
            pass
        caminho_comprovantes = caminho_padrao + 'comprovantes'
        nome_do_arquivo_contador = 1
        contador = 1
        linhas_novo_arquivo = []
        print("  [+] Iniciando a separação de arquivos...")
        for x in arquivos:
            with open(caminho_padrao + x) as arquivo:
                meu_novo_arquivo = []
                print('   [-] Lendo os aquivo(s) TXT(s)')
                for linha in arquivo:
                    linha = linha.strip()
                    linhas_novo_arquivo.append(linha + '\r') #--> cria uma lista de items
                    if 'Codigo de Barras' in linha or len(linha) == 47:

                        if 'Codigo de Barras' in linha:
                            nome_arq = str(linha).replace('Codigo de Barras','').replace(' ','').replace('-','').replace('.','') + '.txt'

                        else:

                            nome_arq = str(linha) + '.txt'

                    if 'Transação efetuada com sucesso por:' in linha:


                        texto = linhas_novo_arquivo
                        texto = texto[13] + texto[14] + '.txt'
                        texto = texto.replace("\nBENEFICIARIO:\n", "")
                        texto = texto.replace("\n", "")
                        texto = texto.replace("\r", "")

                        if texto[0:16] == 'Codigo de Barras':
                            texto = texto.replace(" ", "")
                            texto = texto.replace("-", "")
                            texto = texto.replace("CodigodeBarras", "")


                        elif texto[0:5] == "REMET":

                            texto = 'nao parametrizado' + str(nome_do_arquivo_contador) + '.txt'
                            nome_do_arquivo_contador += 1

                        else :
                            texto = texto.replace("\r", "")
                            texto = texto.replace(" ", "")
                            texto = texto.replace("-", "")
                            texto = texto.replace("CodigodeBarras", "")
                            texto = texto.replace("\n", "")

                        try:
                            with open(caminho_comprovantes + '/'+ nome_arq, 'w') as output:   #--> "w" significa "write" (escrever)
                                output.writelines(linhas_novo_arquivo)
                                linhas_novo_arquivo = []
                                texto = ''
                                nome_arq = ''
                        except:
                            linhas_novo_arquivo = []
                            texto = ''
                            nome_arq = ''

                            pass
        print("   [+] Iniciando o cruzamento de bases...")
        arquivo_listado = os.listdir(caminho_comprovantes)
        obj = {'ID': np.nan,'EMPRESA': np.nan,'NGUIAS':0,'COD': arquivo_listado,'Area': np.nan}
        df2 = pd.DataFrame(data=obj)
        df2['COD'] = df2['COD'].str.replace('.txt','')

        for cdbr in df2.index:
            try:
                Filtro = df[df['COD'].str.contains(df2.COD[cdbr], na = False)]
                df2.loc[cdbr, 'ID'] = Filtro.ID[Filtro.index[0]]
                df2.loc[cdbr, 'EMPRESA'] = Filtro.EMPRESA[Filtro.index[0]]
                df2.loc[cdbr, 'NGUIAS'] = Filtro.NGUIAS[Filtro.index[0]]
                df2.loc[cdbr, 'Area'] = Filtro.Area[Filtro.index[0]]
        

            except:
                Filtro = ''
                pass
            
        df2 = df2.dropna()
        df2['ID'] = df2['ID'].astype(int)
        df2 = df2.reset_index (drop = True)
        tabela_id = df2['ID'].drop_duplicates()
        #Filtro_2 = df2.loc[df2['ID'] == int('17761')].reset_index (drop = True)
        deletar = os.listdir(caminho_comprovantes)
        
        print("   [+] Gerando arquivo PDF...")
        for filtrar in tabela_id.index:
            Filtro_2 = df2.loc[df2['ID'] == int(tabela_id[filtrar])].reset_index (drop = True)
            fileName = caminho_comprovantes + '/'+ str(Filtro_2.ID[0]) + ' - ' + Filtro_2.Area[0] + '.pdf'
            pdf = canvas.Canvas(fileName)
            pdf.setFillColorRGB(0, 0, 0)


            for comprovantes in Filtro_2.index:
                with open(caminho_comprovantes+'/' + Filtro_2.COD[comprovantes] + '.txt') as arquivo:
                    text = pdf.beginText(10, 800)
                    text.setFont("Courier", 11)
                    for linha_ in arquivo:
                        linha_ = linha_.strip()
                        text.textLine(linha_)

                    pdf.drawText(text)
                    pdf.showPage()


            pdf.save()
            

        for dl in deletar:
            try:
                os.remove(caminho_comprovantes + '/' + dl)     
            except:
                pass
        # ak
        relatorio_localizados = df2['ID'].value_counts()
        relatorio_localizados = pd.DataFrame(data=relatorio_localizados)
        relatorio_localizados = relatorio_localizados.rename(columns={'ID': 'NGUIA'})
        #novos_registros.loc[contador_novos_registros, 'Area']
        relatorio_localizados.insert(loc=0, column = 'ID', value = np.nan)
        relatorio_localizados.insert(loc=2, column = 'Area', value = np.nan)
        relatorio_localizados['ID'] = relatorio_localizados.index
        relatorio_localizados = relatorio_localizados.reset_index (drop = True)
        relatorio_total = df
        relatorio_total.insert(loc=4, column = 'LOCALIZADAS', value = 0)
        relatorio_total.insert(loc=5, column = 'DIFERENCA', value = 0)
        for i in relatorio_total.index:
            try:
                filtro3 = relatorio_localizados.loc[relatorio_localizados['ID'] == int(relatorio_total.ID[i])].reset_index (drop = True)
                N_guia = filtro3.NGUIA[0]
                Id_serch = filtro3.ID[0]
                filtro4 = relatorio_total.loc[relatorio_total['ID'] == str(Id_serch)]
                relatorio_total.loc[ filtro4.index,['LOCALIZADAS']] = N_guia

            except:
                filtro3 = ''
                N_guia = ''
                filtro3 = ''
                filtro4 = ''

        del relatorio_total['COD']
        # dropar tudo oq a diferença entre os dois data frames for igual a zero
        relatorio_total["LOCALIZADAS"] = pd.to_numeric(relatorio_total["LOCALIZADAS"])
        relatorio_total["NGUIAS"] = pd.to_numeric(relatorio_total["NGUIAS"])
        relatorio_total['DIFERENCA'] = df.apply(lambda x: x['NGUIAS'] - x['LOCALIZADAS'], axis=1)
        relatorio_total.to_excel(caminho_padrao +'Relatorio {}.xlsx'.format(hoje),index = False)
        #cria um data frame com os nao localizados e deleta do diretorio os que nao localizou, no intuito de 
        #reduzir erros - os que não localizar é necessário fazer a busca manualmente no site do banco
        nao_localizados = relatorio_total[relatorio_total['DIFERENCA'] > 0].reset_index (drop = True)
        arquivo_nao_loc = (caminho_padrao +'Não localizados {}.xlsx'.format(hoje))
        nao_localizados.to_excel(arquivo_nao_loc,index = False)
        for deleta in nao_localizados.index:
            try:
                deletar_arq = caminho_comprovantes +'/'+str(nao_localizados.ID[deleta]) + ' - ' + str(nao_localizados.Area[deleta] + '.pdf')
                os.remove(deletar_arq)
            except:
                pass

        print('  Horário de termino {}:{} - {}/{}/{} '.format(datetime.now().hour, datetime.now().minute,datetime.now().day,datetime.now().month,datetime.now().year ))



    elif event == 'Disponibilizar Comprovantes':
        
        caminho_padrao = (diretorio.split("/"))
        del caminho_padrao[-1]
        separator = '/'
        caminho_padrao = separator.join(caminho_padrao) + separator
        arquivos = os.listdir(caminho_padrao)
        listados = os.listdir(caminho_comprovantes)
        df_comprovantes = pd.DataFrame(data=listados)
        df_comprovantes[['ID','Area']] = df_comprovantes[0].str.split("-",expand=True,)
        comp_tributos = df_comprovantes[df_comprovantes['Area'].str.contains('Tributos', na = False)].reset_index (drop = True)
        comp_dma = df_comprovantes[df_comprovantes['Area'].str.contains('Demais', na = False)].reset_index (drop = True)
        comp_patrimonio = df_comprovantes[df_comprovantes['Area'].str.contains('Patrim', na = False)].reset_index (drop = True)



        caminho_tributos = 'local\folder\ Fiscalizacao\\Attachments\\'
        caminho_patrimonio = 'local\folder\Lists\\Patrimonio\Attachments\\'
        caminho_DMA = 'local\folder\SEGUROS\Attachments\\'

        if len(comp_patrimonio.index) > 0 :
            for x in comp_patrimonio.index:
                fonte = caminho_comprovantes + '/' + (comp_patrimonio[0][x])
                destino = caminho_patrimonio + comp_patrimonio.ID[x].replace(' ','') + '\\'
                shutil.copyfile(fonte, os.path.join(destino,comp_patrimonio[0][x]))
                
            print('Os comprovantes de [Patrimonio] foram disponibilizados.')
            
        else:
            print('Não possui comprovantes em [Patrimonio]')


        if len(comp_tributos.index) > 0 :
            for x in comp_tributos.index:
                fonte = caminho_comprovantes + '/' + (comp_tributos[0][x])
                destino = caminho_tributos + comp_tributos.ID[x].replace(' ','') + '\\'
                shutil.copyfile(fonte, os.path.join(destino,comp_tributos[0][x])) 
                
            print('Os comprovantes de [Tributos] foram disponibilizados.')
            
        else:
            print('Não possui comprovantes em [Tributos]')

        if len(comp_dma.index) > 0 :
            for x in comp_dma.index:
                fonte = caminho_comprovantes + '/' + (comp_dma[0][x])
                destino =caminho_DMA + comp_dma.ID[x].replace(' ','') + '\\'
                shutil.copyfile(fonte, os.path.join(destino,comp_dma[0][x])) 
                
            print('Os comprovantes das [Demais Áreas] foram disponibilizados.')
            
        else:
            print('Não possui comprovantes em [Demais Áreas]')
            
        print('A entrega de comprovantes foi finalizada!')
        
        
        
    elif event == 'Instruções': 
        print('Para utilizar o programa de análise das bases, siga as seguintes instruções :\n • Os arquivo em excel deve estar no formato XLSX. \n • Insira os arquivo aonde indica "Planilha Shaepoint".  \n • Em seguida no próximo campo selecione a pasta aonde estão salvos\n   os comprovantes em txt.\n ')


    else:
        break
        window.close()